In [1]:
!pip install transformers[torch] -q

In [2]:
!pip install datasets -q --upgrade

In [3]:
!pip install evaluate -q

In [4]:
!pip install wandb -q --upgrade

In [5]:
import os
import gc
import math
import librosa
import numpy as np
import pandas as pd
import librosa as lb
import matplotlib.pyplot as plt

from scipy import signal
from scipy.fft import fftshift

from collections import defaultdict
from itertools import islice
from typing import Any
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, precision_score, recall_score

import torch
import torchaudio

from IPython import display
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset
from tqdm.auto import trange
from tqdm import tqdm
tqdm.pandas()

import re
import seaborn as sns
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import (
    load_dataset,
    load_metric,
    load_from_disk,
    Audio,
    concatenate_datasets
)
from transformers import (
    AutoConfig,
    Wav2Vec2Processor,
    Wav2Vec2Tokenizer,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2ForSequenceClassification,
    Wav2Vec2Model,
    TrainingArguments,
    Trainer,
    get_linear_schedule_with_warmup,
    EvalPrediction
)

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)
import evaluate

2024-05-26 20:56:14.868154: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 20:56:14.868249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 20:56:14.996242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
import os
os.environ["WANDB_PROJECT"]="thesis_model_batch"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [8]:
import datasets
datasets.__version__

'2.19.1'

In [9]:
train_path = '/kaggle/input/thesis-train-data/train_one'

train1_path = '/kaggle/input/thesis-model-batch-3sec-8/train1'
train2_path = '/kaggle/input/thesis-model-batch-3sec-8/train2'

val_path = '/kaggle/input/thesis-train-data/val_one.pickle'
path_to_save = '/kaggle/working/second_model.pth'

model_name = 'facebook/wa|v2vec2-large-xlsr-53'
run_name = 'basic_model_one_tahn_russian'

In [10]:
from datasets import concatenate_datasets
train1 = load_from_disk(train1_path)
train2 = load_from_disk(train2_path)

train = concatenate_datasets([train1, train2])
val_pd = pd.read_pickle(val_path)

Loading dataset from disk:   0%|          | 0/24 [00:00<?, ?it/s]

In [12]:
class CustomDataset(Dataset):

    def __init__(self, input_values, labels, audio_paths):
        super().__init__()
        self.input_values = input_values
        self.labels = labels
        self.audio_paths = audio_paths

    def __len__(self):
        return len(self.input_values)

    def __getitem__(self, idx):
        return {'input_values': self.input_values[idx].tolist(), 'labels': self.labels[idx], 'audio_ID': self.audio_paths[idx]}

In [13]:
val = CustomDataset(
    input_values = val_pd['input_values'].tolist(),
    labels = val_pd['labels'].tolist(),
    audio_paths = val_pd['audio_ID'].tolist()
)

In [14]:
import gc
del val_pd
gc.collect()

1259

## Initialize model

In [20]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [35]:
# https://github.com/m3hrdadfi/soxan/blob/main/notebooks/Emotion_recognition_in_Greek_speech_using_Wav2Vec2.ipynb
import torch
from torch import nn
from torch.nn import functional as nnf
from tqdm import tqdm
tqdm.pandas()

from transformers import (
    Wav2Vec2Model
)
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.lstm = nn.LSTM(
            config['input_size'],
            config['hidden_size'],
            num_layers = 2,
            bidirectional = True,
            dropout = 0.1,
            batch_first = True
        )
        
        self.fc = nn.Linear(config['hidden_size'] * 2,config['num_labels'])
    
    def forward(self,input_values):     
        packed_output,(hidden_state,cell_state) = self.lstm(input_values)
        hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1) 
        dense_outputs=self.fc(hidden)
        return dense_outputs

class Wav2VecClassifer(Wav2Vec2PreTrainedModel):

    def __init__(self, w2v_config, config):
        super().__init__(w2v_config)

        self.num_labels = config['num_labels']
        self.pooling_strategy = config['pooling_mode']
        self.wav2vec2 = Wav2Vec2Model.from_pretrained(
            config['model_name'], 
            config=w2v_config
        )
        self.proj_layer = nn.Linear(config['input_size'], 256)
        self.classifier = Wav2Vec2ClassificationHead(config)
        self.loss = nn.CrossEntropyLoss()
        self.post_init()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            labels,
            attention_mask=None,
            device=None,
            output_attentions=False,
            output_hidden_states=False,
            number_of_chunks=None
    ):

#         number_of_chunks = [len(x) for x in input_values]

        outputs = self.wav2vec2(
            input_values=input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states
        )
        hidden_states = outputs.last_hidden_state
        all_hs = []
        for hs in hidden_states:
            all_hs.append(hs.view(1, -1, 1))
        chunks_num = len(all_hs) - 1

        concat_states = torch.cat(all_hs, dim=2)
        raw = nnf.fold(concat_states, (249+100*chunks_num, 1024), kernel_size=(249, 1024), stride=(100, 1024))
        counter = nnf.fold(torch.ones_like(concat_states), (249+100*chunks_num, 1024), kernel_size=(249, 1024), stride=(100, 1024))
        result = raw / counter

        pooled_preds = self.classifier(result.squeeze(0))

        loss = None
        if labels is not None:
            loss = self.loss(pooled_preds.view(-1, 2), labels.view(-1))
        output = (pooled_preds,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [36]:
config = {
    'output_hidden_states': False,
    'num_labels': 2,
    'hidden_size': 64,
    'final_dropout': 0.1,
    'hidden_dropout': 0.2,
    'pooling_mode': 'mean',
    'model_name': 'jonatasgrosman/wav2vec2-large-xlsr-53-russian',
    'input_size': 1024
}

model_args_w2v2 = {
    "attention_dropout": 0.1,
    "hidden_dropout": 0.1,
    "feat_proj_dropout": 0.0,
    "mask_time_prob": 0.05,
    "layerdrop": 0.1,
    'layer_norm_eps':1e-4,
}

model_args_w2v2_config = Wav2Vec2Config.from_pretrained(
    config['model_name'], 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
#     layer_norm_eps:1e-4,
)

model = Wav2VecClassifer(model_args_w2v2_config, config).cuda()

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
model.freeze_feature_extractor()

### Trainer

In [39]:
from torch import Tensor

def collate_fn_pooled_tokens(data):
    input_ids = [torch.tensor(data[i]['input_values']) for i in range(len(data))]
    attention_mask = [torch.where(input_ids[i]==0, 0, 1) for i in range(len(input_ids))]
    label_features = [data[i]['labels'] for i in range(len(data))]
    d_type = torch.long if isinstance(label_features[0], int) else torch.float
    if len(data[0]) == 2:
        collated = {
            'input_values': input_ids[0],
            'attention_mask': attention_mask[0],
            'labels': torch.tensor(label_features, dtype=d_type)
        }
    else:
        collated = {
            'input_values': input_ids[0],
            'attention_mask': attention_mask[0],
            'labels': torch.tensor(label_features, dtype=d_type)
        }
    return collated

In [40]:
processor = Wav2Vec2FeatureExtractor.from_pretrained('facebook/wav2vec2-large-xlsr-53')
data_collator = collate_fn_pooled_tokens

In [42]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    labels = p.label_ids

    cr = evaluate.load('bstrai/classification_report')
    result = cr.compute(predictions=preds, references=labels, labels=[0, 1])
    
    res_dict = {}
    for label in result.keys():
        if isinstance(result[label], dict):
            for metric in result[label].keys():
                res_dict[f'{metric}_{label}'] = result[label][metric]
        else:
            res_dict[label] = result[label]

    return res_dict

In [43]:
import random
def set_seed(seed: int, deterministic: bool = False):
        """
        Helper function for reproducible behavior to set the seed in `random`, `numpy`, `torch` and/or `tf` (if installed).

        Args:
            seed (`int`):
                The seed to set.
            deterministic (`bool`, *optional*, defaults to `False`):
                Whether to use deterministic algorithms where available. Can slow down training.
        """
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
        if deterministic:
            torch.use_deterministic_algorithms(True)

set_seed(88) #888

In [44]:
from transformers import TrainingArguments

In [45]:
num_epochs = 2
warmup_ratio = 0.1
total_steps = len(train) * num_epochs

In [46]:
total_steps

74556

In [ ]:
lr = 1e-3
config = {
    'output_hidden_states': False,
    'num_labels': 2,
    'hidden_size': 64,
    'final_dropout': 0.1,
    'hidden_dropout': 0.2,
    'pooling_mode': 'mean',
    'model_name': 'jonatasgrosman/wav2vec2-large-xlsr-53-russian',
    'input_size': 1024
}

model_args_w2v2 = {
    "attention_dropout": 0.1,
    "hidden_dropout": 0.1,
    "feat_proj_dropout": 0.0,
    "mask_time_prob": 0.05,
    "layerdrop": 0.1,
    'layer_norm_eps':1e-4,
}

model_args_w2v2_config = Wav2Vec2Config.from_pretrained(
    config['model_name'], 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
#     layer_norm_eps:1e-4,
)

model = Wav2VecClassifer(model_args_w2v2_config, config).cuda()
model.freeze_feature_extractor()

optimizer = torch.optim.AdamW([
        {"params": model.wav2vec2.parameters(), "lr": 3e-4},
    ], lr=lr)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps * warmup_ratio,
    num_training_steps=total_steps
)

training_args = TrainingArguments(
    output_dir=f"/content/model_one_trainer_{lr}",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=num_epochs,
    fp16=True,
    save_steps=3000,
    eval_steps=1500,
    learning_rate=lr,
    save_total_limit=3,
    report_to='wandb',
    run_name=f"model_2_{lr}_w2v_3e-4_lstm_one_segment",
    logging_steps=1500,
    load_best_model_at_end=True,
    seed=888
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=processor,
    optimizers=(optimizer, scheduler)
)

trainer.train()